In [1]:
import os, shutil, random, glob
import cv2
import numpy as np
import pandas as pd
import keras
from keras.utils import plot_model  
from keras.applications.resnet50 import ResNet50  
from keras.applications.vgg16 import VGG16  
from keras.applications.inception_v3 import InceptionV3  
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from keras.models import Model, load_model  
from keras.optimizers import SGD  
from keras.preprocessing.image import ImageDataGenerator  
import matplotlib.pyplot as plt  

Using TensorFlow backend.


In [2]:
resize = 224
def load_data():
    imgs = os.listdir("./train/")
    num = len(imgs)
    train_data = np.empty((500, resize, resize, 3), dtype="int32")
    train_label = np.empty((500, ), dtype="int32")
    test_data = np.empty((500, resize, resize, 3), dtype="int32")
    test_label = np.empty((500, ), dtype="int32")
    for i in range(500):
        if i % 2:
            train_data[i] = cv2.resize(cv2.imread('./train/' + 'dog.' + str(i) + '.jpg'), (resize, resize))
            train_label[i] = 1
        else:
            train_data[i] = cv2.resize(cv2.imread('./train/' + 'cat.' + str(i) + '.jpg'), (resize, resize))
            train_label[i] = 0
    for i in range(500, 1000):
        if i % 2:
            test_data[i-500] = cv2.resize(cv2.imread('./train/' + 'dog.' + str(i) + '.jpg'), (resize, resize))
            test_label[i-500] = 1
        else:
            test_data[i-500] = cv2.resize(cv2.imread('./train/' + 'cat.' + str(i) + '.jpg'), (resize, resize))
            test_label[i-500] = 0
    return train_data, train_label, test_data, test_label

In [3]:
train_data, train_label, test_data, test_label = load_data()
train_data, test_data = train_data.astype('float32'), test_data.astype('float32')
train_data, test_data = train_data/255, test_data/255

In [4]:
train_label = keras.utils.to_categorical(train_label, 2)
test_label = keras.utils.to_categorical(test_label, 2)

In [5]:
base_model = VGG16(weights='imagenet', include_top=False, pooling=None, input_shape=(resize, resize, 3), classes = 2)
for layer in base_model.layers:
    layer.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [7]:
model.fit(train_data, train_label,
          batch_size = 64,
          epochs = 50,
          validation_split = 0.2,
          shuffle = True)

Train on 400 samples, validate on 100 samples
Epoch 1/50
400/400 [==============================] - 640s 2s/step - loss: 0.1722 - acc: 0.9537 - val_loss: 0.1961 - val_acc: 0.9500
Epoch 2/50
400/400 [==============================] - 702s 2s/step - loss: 0.1637 - acc: 0.9475 - val_loss: 0.1946 - val_acc: 0.9450
Epoch 3/50
400/400 [==============================] - 673s 2s/step - loss: 0.1827 - acc: 0.9375 - val_loss: 0.2009 - val_acc: 0.9150
Epoch 4/50
400/400 [==============================] - 594s 1s/step - loss: 0.1458 - acc: 0.9563 - val_loss: 0.2060 - val_acc: 0.9050
Epoch 5/50
400/400 [==============================] - 555s 1s/step - loss: 0.1718 - acc: 0.9275 - val_loss: 0.2248 - val_acc: 0.9300
Epoch 6/50
400/400 [==============================] - 549s 1s/step - loss: 0.1610 - acc: 0.9475 - val_loss: 0.1925 - val_acc: 0.9500
Epoch 7/50
400/400 [==============================] - 552s 1s/step - loss: 0.1434 - acc: 0.9612 - val_loss: 0.1933 - val_acc: 0.9500
Epoch 8/50
400/400 [===

In [8]:
scores = model.evaluate(train_data, train_label, verbose = 1)
print(scores)

500/500 [==============================] - 299s 597ms/step
[0.08602091193199157, 0.972]


In [9]:
scores = model.evaluate(test_data, test_label, verbose = 1)
print(scores)

500/500 [==============================] - 303s 606ms/step
[0.23462430983781815, 0.8979999995231629]
